In [11]:
import h5py
import numpy as np
import tensorflow as tf
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_score, recall_score, fbeta_score
from tensorflow.keras.models import clone_model
from tqdm import tqdm
import pathlib
import sys
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
attack_vector='cache' #network
machine_setting='dynamic_browser_version' # static_browser_version

root_path=f'/content/drive/MyDrive/Colab Notebooks/concept-drift-online/'
data_path=f'{root_path}/data/{machine_setting}/{attack_vector}'


In [4]:
os.getcwd()

'/content'

In [10]:
sys.path.append(str(root_path)) # Add the directory to the path
from utils import prepare_hdf5_data,predict_on_hdf5,get_data_paths_ordered,OpenWorldLabelEncoder

In [6]:

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [7]:
model_path=f'{data_path}/models/cnn_lstm_cache.keras'
le_dir=f'{data_path}/models/label_encoder_100.pkl'
model0 = tf.keras.models.load_model(model_path,)
with open(le_dir, 'rb') as f:
    le = pickle.load(f)

In [8]:
ordered_paths=get_data_paths_ordered(data_path)


In [9]:
batch=256
max_epochs=10
n_incremental_samples_per_website=10

results={}
for path in tqdm(ordered_paths):
    curr_date_i=path.stem
    train_path_i = (path/'train.h5').as_posix()
    test_path_i = (path/'test.h5').as_posix()
    train_i=prepare_hdf5_data(train_path_i,le,batch_size=batch,n_samples=n_incremental_samples_per_website,sample_validation=-1,verbose=0)['data']
    model_i=clone_model(model0)
    model_i.set_weights(model0.get_weights())
    model_i.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),loss='categorical_crossentropy',  metrics=['accuracy'])
    model_i.fit(train_i,epochs=max_epochs,verbose=0)
    pred_i=predict_on_hdf5(test_path_i,model_i,le,batch_size=32)
    with h5py.File(test_path_i, 'r') as hdf:
      y_testi = le.transform((hdf[f'site_name'][:]).reshape(-1,1)).toarray()
    acci=(y_testi.argmax(axis=1)==pred_i.argmax(axis=1)).mean()
    results.update({curr_date_i:acci})
    print(f'accuracy: {acci}')

  0%|          | 0/7 [00:00<?, ?it/s]

32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step


 14%|█▍        | 1/7 [00:47<04:42, 47.02s/it]

accuracy: 0.915
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step


 29%|██▊       | 2/7 [01:21<03:18, 39.64s/it]

...

accuracy: 0.735
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step


 86%|████████▌ | 6/7 [03:23<00:32, 32.59s/it]

accuracy: 0.734
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step


100%|██████████| 7/7 [03:53<00:00, 33.29s/it]

accuracy: 0.687


In [14]:
results={}
batch_size=256
non_sensitive_label=69
owle = OpenWorldLabelEncoder(le,non_sensitive_label+1)

ow_model_path=f'{data_path}/models/cnn_lstm_ow.keras'
model_ow = tf.keras.models.load_model(ow_model_path)
for path in tqdm(ordered_paths):
    curr_date_i=path.stem
    train_path_i = (path/'train.h5').as_posix()
    test_path_i = (path/'test.h5').as_posix()
    train_i=prepare_hdf5_data(train_path_i,owle,batch_size=batch,n_samples=n_incremental_samples_per_website,sample_validation=-1,verbose=0)['data']
    model_i=clone_model(model_ow)
    model_i.set_weights(model_ow.get_weights())
    model_i.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),loss='categorical_crossentropy',  metrics=['accuracy'])
    model_i.fit(train_i,epochs=max_epochs,verbose=0)
    test_datasets_i=prepare_hdf5_data(test_path_i,owle,batch_size=batch,sample_validation=-1)['data']
    all_predictions = []
    all_true_labels = []
    for features, labels in test_datasets_i:
        batch_predictions = model_i.predict_on_batch(features)
        all_predictions.append(batch_predictions)
        all_true_labels.append(labels.numpy()) # Convert label tensor to NumPy array
    pred_i = np.concatenate(all_predictions, axis=0)
    y_test_aligned = np.concatenate(all_true_labels, axis=0)
    acci=(pred_i.argmax(axis=1)==y_test_aligned.argmax(axis=1)).mean()
    y_true_indices = y_test_aligned.argmax(axis=1)
    y_pred_indices = pred_i.argmax(axis=1)
    precision = precision_score(y_true_indices, y_pred_indices, labels=[non_sensitive_label], average='weighted')
    recall = recall_score(y_true_indices, y_pred_indices, labels=[non_sensitive_label], average='weighted')
    f2score = fbeta_score(y_true_indices, y_pred_indices, beta=2, labels=[non_sensitive_label], average='weighted')

    results.update({curr_date_i: {'accuracy': acci, 'precision': precision, 'recall': recall, 'f2score': f2score}})
    print(f'Date: {curr_date_i}, Accuracy: {acci}, Precision (Non-sensitive): {precision}, Recall (Non-sensitive): {recall}, F2-score (Non-sensitive): {f2score}')


 14%|█▍        | 1/7 [00:38<03:51, 38.53s/it]

Date: 2021_08_27, Accuracy: 0.916, Precision (Non-sensitive): 0.9475409836065573, Recall (Non-sensitive): 0.932258064516129, F2-score (Non-sensitive): 0.9352750809061489


 29%|██▊       | 2/7 [01:02<02:28, 29.72s/it]

Date: 2021_09_05, Accuracy: 0.877, Precision (Non-sensitive): 0.9129032258064517, Recall (Non-sensitive): 0.9129032258064517, F2-score (Non-sensitive): 0.9129032258064517


 43%|████▎     | 3/7 [01:22<01:41, 25.47s/it]

...

Date: 2021_11_28, Accuracy: 0.723, Precision (Non-sensitive): 0.7485207100591716, Recall (Non-sensitive): 0.8161290322580645, F2-score (Non-sensitive): 0.8016476552598225


 86%|████████▌ | 6/7 [02:24<00:21, 21.77s/it]

Date: 2022_01_23, Accuracy: 0.704, Precision (Non-sensitive): 0.7629179331306991, Recall (Non-sensitive): 0.8096774193548387, F2-score (Non-sensitive): 0.7998725302740599


100%|██████████| 7/7 [02:41<00:00, 23.13s/it]

Date: 2022_02_21, Accuracy: 0.68, Precision (Non-sensitive): 0.6751412429378532, Recall (Non-sensitive): 0.7709677419354839, F2-score (Non-sensitive): 0.7496863237139272


In [15]:
results

{'2021_08_27': {'accuracy': np.float64(0.916),
  'precision': 0.9475409836065573,
  'recall': 0.932258064516129,
  'f2score': 0.9352750809061489},
 '2021_09_05': {'accuracy': np.float64(0.877),
  'precision': 0.9129032258064517,
  'recall': 0.9129032258064517,
  'f2score': 0.9129032258064517},
 '2021_09_12': {'accuracy': np.float64(0.855),
  'precision': 0.9197324414715721,
  'recall': 0.8870967741935484,
  'f2score': 0.8934372969460689},
 '2021_10_24': {'accuracy': np.float64(0.698),
  'precision': 0.7881619937694704,
  'recall': 0.8161290322580645,
  'f2score': 0.8103779628443306},
 '2021_11_28': {'accuracy': np.float64(0.723),
  'precision': 0.7485207100591716,
  'recall': 0.8161290322580645,
  'f2score': 0.8016476552598225},
 '2022_01_23': {'accuracy': np.float64(0.704),
  'precision': 0.7629179331306991,
  'recall': 0.8096774193548387,
  'f2score': 0.7998725302740599},
 '2022_02_21': {'accuracy': np.float64(0.68),
  'precision': 0.6751412429378532,
  'recall': 0.7709677419354839,
